<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/Streamlit_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install necessary packages:**

In [ ]:
!pip install PyPDF2 tabulate streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=a6669bdc2f6788e602b1ba3a50e2c5d86c8de2d50aa70dcbc59b44ccb49c7390
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [ ]:
!pip install pandas openpyxl


In [ ]:
!pip install altair


In [ ]:
!pip install Whoosh


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 5.8 MB/s eta 0:00:00


# **Modify the code for Streamlit**

In [24]:
%%writefile streamlit_app.py

import streamlit as st
import pandas as pd
import PyPDF2
import re
import os
import altair as alt
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.writing import BufferedWriter
from whoosh.qparser import QueryParser
from io import BytesIO
import base64

# Create a function to generate download links
def to_excel(df):
    output = BytesIO()
    writer = pd.ExcelWriter(output, engine='openpyxl')
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    writer.save()
    processed_data = output.getvalue()
    return processed_data

def get_table_download_link(df, filename='data.xlsx', text='Download Excel file'):
    val = to_excel(df)
    b64 = base64.b64encode(val)
    return f'<a href="data:application/octet-stream;base64,{b64.decode()}" download="{filename}">{text}</a>'

# App Title
st.title("PDF Data Extraction Tool")

# Indexing the text for full-text search
schema = Schema(title=TEXT(stored=True), content=TEXT(stored=True))
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = create_in("indexdir", schema)

uploaded_files = st.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)

results = []
headers = ["File Name", "C.I.L Number", "Total à Payer (Dirhams)", "Date Limite de Paiement", "Numéro de Client"]

if uploaded_files:
    writer = BufferedWriter(ix)
    for uploaded_file in uploaded_files:
        pdf_reader = PyPDF2.PdfReader(uploaded_file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

        writer.add_document(title=uploaded_file.name, content=text)

        cil_pattern = r"(\d{7}-\d{3})"
        total_payer_pattern = r"(\d{3,4},\d{2})"
        date_limite_pattern = r"(\d{2}-\d{2}-\d{4})"

        format_matches = re.findall(cil_pattern, text)
        total_payer_match = re.search(total_payer_pattern, text)
        date_limite_match = re.search(date_limite_pattern, text)

        cil_number = format_matches[-1] if len(format_matches) > 1 else "Not found"
        numero_client = format_matches[0] if len(format_matches) > 1 and format_matches[0] != cil_number else "Not found"
        total_payer = total_payer_match.group(1) if total_payer_match else "Not found"
        date_limite = date_limite_match.group(1) if date_limite_match else "Not found"

        results.append([uploaded_file.name, cil_number, total_payer, date_limite, numero_client])

    writer.commit()
    writer.close()

    df = pd.DataFrame(results, columns=headers)

    # Display data in a table format
    st.write(df)

    # Generate download link for Excel
    st.markdown(get_table_download_link(df), unsafe_allow_html=True)

    # Convert data types
    df['Date Limite de Paiement'] = pd.to_datetime(df['Date Limite de Paiement'], format='%d-%m-%Y', errors='coerce')
    df['Total à Payer (Dirhams)'] = df['Total à Payer (Dirhams)'].str.replace(',', '.').astype(float)

    # Create the Altair chart
    chart = alt.Chart(df).mark_bar().encode(
        x='Date Limite de Paiement:T',
        y='Total à Payer (Dirhams):Q',
        color='File Name:N',
        tooltip=['File Name', 'Date Limite de Paiement', 'Total à Payer (Dirhams)']
    )
    st.altair_chart(chart, use_container_width=True)

# Full-text search
search_term = st.text_input("Search in extracted values to find PDFs")
if search_term:
    with ix.searcher() as searcher:
        query = QueryParser("content", ix.schema).parse(search_term)
        results = searcher.search(query, limit=None)
        for hit in results:
            st.write(f"PDF Name: {hit['title']}")


Overwriting streamlit_app.py


# **Run Streamlit on Google Colab**

In [25]:
import os

# Download and unzip ngrok only if it doesn't exist yet
if not os.path.exists("ngrok-stable-linux-amd64.zip"):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip -o ngrok-stable-linux-amd64.zip

# Authenticate with ngrok using your auth token
!./ngrok authtoken 2Wtkkzkovj51i2fQ8MUCQrf7DyT_7cft4ztcihL3M9MvpsKdC


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [26]:
# Start ngrok for the Streamlit port 8501
get_ipython().system_raw('./ngrok http 8501 &')
!sleep 2  # Wait a couple of seconds for the tunnel to be established

# Fetch and print the public URL
public_url = !curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; tunnels = json.load(sys.stdin)['tunnels']; print(tunnels[0]['public_url'] if tunnels else '')"
print(public_url[0])


https://8663-34-83-49-196.ngrok-free.app


In [ ]:
!streamlit run streamlit_app.py





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.49.196:8501

/content/streamlit_app.py:20: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/content/streamlit_app.py:20: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/content/streamlit_app.py:20: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
/content/streamlit_app.py:20: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
2023-10-18 15:25:54.924 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.p